In [1]:
%matplotlib inline
import pandas as pd
import datetime
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch.utils.data.dataloader import default_collate
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from IPython.display import clear_output
from torch.autograd import Variable
print(torch.__version__)

1.0.0


In [2]:
class IMUDataset(Dataset):
    def __init__(self, mode = 'test', transform = None):
        if mode == 'train' :
            self.df = pd.read_csv('data_preparation/train.csv', header = 0, index_col = 0)
        elif mode == 'test' :
            self.df = pd.read_csv('data_preparation/test.csv', header = 0, index_col = 0)
        elif mode == 'val' :
            self.df = pd.read_csv('data_preparation/val.csv', header = 0, index_col = 0)
        self.transform = transform
        self.df = self.df.reset_index()
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        y = self.df.iloc[idx : idx + 80, 3 : ].values
        ind = np.argmax(np.sum(y, axis = 0))
        label = np.zeros_like(self.df.iloc[0, 3 : ].values)
        label = label.astype('float')
        label[ind] = 1
        x = self.df.iloc[idx : idx + 80, : 3].values
        x = x.astype('float')
        assert(x.shape == (80, 3))
        assert(label.shape == (6, ))
        return x, label
        
trainset = IMUDataset(mode = 'train')
valset = IMUDataset(mode = 'val')
testset = IMUDataset(mode = 'test')

In [3]:
batch_size = 256
def my_collate(batch):
    "Puts each data field into a tensor with outer dimension batch size"
    batch = list(filter(lambda x : x is not None, batch))
    return default_collate(batch)

train_indices = [(i * 80) for i in range(len(trainset) // 80)]
val_indices = [(i * 80) for i in range(len(valset) // 80)]
test_indices = [(i * 80) for i in range(len(testset) // 80)]

trainloader = DataLoader(trainset, batch_size = batch_size, sampler = SubsetRandomSampler(train_indices), collate_fn = my_collate, drop_last = True)
valloader = DataLoader(valset, batch_size = batch_size, sampler = SubsetRandomSampler(val_indices), collate_fn = my_collate, drop_last = True)
testloader = DataLoader(testset, batch_size = batch_size, sampler = SubsetRandomSampler(test_indices), collate_fn = my_collate, drop_last = True)

# for idx, (x, y) in enumerate(trainloader) :
#     print(x.shape, y.shape)

In [10]:
class ConvAutoEncoder(nn.Module):
    def __init__(self, code_size):
        super(ConvAutoEncoder, self).__init__()
        self.code_size = code_size
        # defining layers
        # input size - batch_size x 3 x 100
        conv1 = nn.Conv1d(3, 10, 3)
        # after conv1 - batch_size x 10 x 98
        conv2 = nn.Conv1d(10, 20, 1)
        # after conv2 - batch_size x 10 x 98
        enc_fc = nn.Linear(10 * 98, self.code_size)
        
        dec_fc = nn.Linear(self.code_size, 10 * 98)
        deconv2 = nn.ConvTranspose1d(20, 10, 1)
        deconv1 = nn.ConvTranspose1d(10, 3, 3)
        
    def forward(self, signal):
        code = self.encode(signal)
        out = self.decode(code)
        return out, code
    
    def encode(signal):
        signal = signal.view(16, 3, 80)
        code = F.sigmoid(self.conv1(signal))
        code = F.sigmoid(self.conv2(code))
        code = F.sigmoid(self.enc_fc(code))
        return code
    
    def decode(code):
        out = F.sigmoid(self.dec_fc(code))
        out = F.sigmoid(self.deconv2(out))
        out = F.sigmoid(self.deconv1(out))
        return out
    
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # defining layers
        self.conv1 = nn.Conv1d(3, 5, 3)
        self.conv2 = nn.Conv1d(5, 10, 5)
        self.conv3 = nn.Conv1d(10, 10, 5)
        self.fc = nn.Linear(70 * 10, 6)
        
        nn.init.xavier_uniform_(self.conv1.weight, gain = nn.init.calculate_gain('relu'))
        nn.init.xavier_uniform_(self.conv2.weight, gain = nn.init.calculate_gain('relu'))
        nn.init.xavier_uniform_(self.conv3.weight, gain = nn.init.calculate_gain('relu'))
        nn.init.xavier_uniform_(self.conv3.weight, gain = nn.init.calculate_gain('sigmoid'))
        
    def forward(self, signal):
        signal = signal.view(batch_size, 3, 80)
        out = F.relu(self.conv1(signal))
        out = F.relu(self.conv2(out))
        out = F.relu(self.conv3(out))
        out = out.view(-1, 70 * 10)
        out = self.fc(out)
        out = F.log_softmax(out, dim = 1)
        return out
    
Net = ConvNet()
if torch.cuda.is_available():
    Net = Net.cuda()

In [5]:
def output_size(n, f, p = 0, s = 1):
    return (((n + 2 * p - f) / s) + 1)

print(output_size(80, 3))
print(output_size(78, 5))
print(output_size(74, 5))

78.0
74.0
70.0


In [21]:
import torch.optim as optim
optimizer = optim.Adam(Net.parameters(), lr=1e-2)

In [22]:
num_epochs = 10
total_step = len(trainset) // 80
train_loss_list = list()
val_loss_list = list()
for epoch in range(num_epochs):
    trn = []
    for i, (images, labels) in enumerate(trainloader):
        images = Variable(images).cuda().float()
        labels = Variable(labels).cuda()
        _, target = torch.max(labels, 1)

        y_pred = Net(images)

        loss = F.cross_entropy(y_pred, target)
        trn.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#         clear_output(wait=True)
#         print(f'epoch: {epoch}, step : {i * len(images)} of {total_step}, loss : {loss}')
        if i % 10 == 0 :
            print('epoch = ', epoch, ' step = ', i * len(images), ' of total steps ', total_step, ' loss = ', loss.item())
    train_loss = (sum(trn) / len(trn))
    train_loss_list.append(train_loss)
    print('Training Loss : ', train_loss)
    
    val = []
    print('validating....')
    for i, (images, labels) in enumerate(valloader):
        images = Variable(images).cuda().float()
        labels = Variable(labels).cuda()
        _, target = torch.max(labels, 1)

        # Forward pass
        outputs = Net(images)
#         loss = criterion(outputs.float(), labels.float())
        loss = F.cross_entropy(outputs, target)
        val.append(loss)
#     print(f'validation loss : {sum(val)/len(val)}')
    val_loss = (sum(val) / len(val)).item()
    val_loss_list.append(val_loss)
    print('Validation Loss : ', val_loss)

epoch =  0  step =  0  of total steps  9792  loss =  1.4034284353256226
epoch =  0  step =  2560  of total steps  9792  loss =  1.4118857383728027
epoch =  0  step =  5120  of total steps  9792  loss =  1.3866299390792847
epoch =  0  step =  7680  of total steps  9792  loss =  1.497254490852356
Training Loss :  1.4322597509936283
validating....
Validation Loss :  1.4125292301177979
epoch =  1  step =  0  of total steps  9792  loss =  1.35170578956604
epoch =  1  step =  2560  of total steps  9792  loss =  1.373122215270996
epoch =  1  step =  5120  of total steps  9792  loss =  1.47686767578125
epoch =  1  step =  7680  of total steps  9792  loss =  1.4345091581344604
Training Loss :  1.425133880816008
validating....
Validation Loss :  1.4054722785949707
epoch =  2  step =  0  of total steps  9792  loss =  1.4417718648910522
epoch =  2  step =  2560  of total steps  9792  loss =  1.3141549825668335
epoch =  2  step =  5120  of total steps  9792  loss =  1.415033221244812
epoch =  2  st

KeyboardInterrupt: 

In [11]:
a = next(iter(trainloader))
print(torch.max(a[1], 1)[1])

tensor([5, 5, 5, 2, 4, 4, 5, 5, 4, 4, 5, 5, 0, 1, 5, 5, 5, 1, 1, 0, 4, 5, 5, 5,
        5, 5, 4, 5, 5, 1, 0, 1, 1, 5, 0, 5, 0, 4, 0, 4, 1, 1, 0, 5, 5, 0, 5, 5,
        5, 5, 3, 1, 1, 1, 5, 5, 5, 1, 3, 1, 1, 5, 5, 5, 5, 1, 1, 1, 5, 5, 1, 1,
        0, 1, 5, 5, 5, 1, 4, 5, 1, 0, 4, 5, 1, 5, 3, 0, 1, 1, 1, 5, 5, 2, 5, 4,
        5, 5, 1, 1, 0, 5, 1, 5, 0, 0, 5, 5, 5, 5, 0, 5, 1, 1, 1, 5, 1, 4, 0, 5,
        5, 1, 5, 1, 5, 5, 1, 1, 4, 2, 4, 5, 1, 5, 1, 5, 5, 5, 1, 4, 1, 5, 5, 0,
        5, 5, 5, 1, 4, 4, 1, 1, 5, 1, 5, 5, 5, 2, 5, 1, 1, 4, 5, 5, 5, 5, 5, 5,
        5, 5, 4, 5, 5, 4, 5, 1, 1, 4, 1, 5, 1, 1, 1, 0, 5, 3, 1, 3, 5, 0, 5, 5,
        1, 2, 4, 0, 5, 1, 1, 4, 2, 1, 4, 5, 5, 0, 1, 5, 5, 1, 0, 4, 5, 4, 5, 5,
        2, 1, 5, 1, 2, 1, 5, 5, 5, 4, 4, 1, 1, 4, 1, 5, 1, 4, 0, 5, 1, 1, 1, 4,
        5, 5, 1, 5, 1, 0, 5, 4, 5, 2, 5, 5, 5, 1, 5, 4])
